# Following below youtube tutorial

### Youtube video : https://www.youtube.com/watch?v=QIUxPv5PJOY&ab_channel=ComputerScience

### Going to analyse for indian stock : TATA MOTORS


* ### It is mentioned in the medium article : https://randerson112358.medium.com/stock-price-prediction-using-python-machine-learning-e82a039ac2bb that it uses Long Short-Term Memory (LSTM)


* ### Good video on LSTM : https://www.youtube.com/watch?v=WCUNPb-5EYI&ab_channel=BrandonRohrer


In [4]:
#Import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [7]:
#Get the stock quote 
# we can simply get the name form YAhoo site : https://in.finance.yahoo.com/quote/TATAMOTORS.NS/

df = web.DataReader('TATAMOTORS.NS', data_source='yahoo', start='2017-01-01', end='2020-10-09') 
#Show the data 
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-02,488.700012,469.049988,472.000000,487.049988,4024156.0,487.049988
2017-01-03,492.700012,479.899994,491.000000,481.100006,5814518.0,481.100006
2017-01-04,493.700012,484.100006,484.649994,486.649994,4769872.0,486.649994
2017-01-05,503.399994,495.549988,496.000000,502.149994,7961242.0,502.149994
2017-01-06,507.950012,496.250000,502.950012,497.750000,4907475.0,497.750000
...,...,...,...,...,...,...
2020-10-05,137.500000,132.050003,136.949997,133.899994,55268322.0,133.899994
2020-10-06,145.800003,136.000000,138.000000,144.750000,161987221.0,144.750000
2020-10-07,144.500000,139.800003,144.250000,141.000000,68273641.0,141.000000
